In [51]:
import pandas as pd
from keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dropout
from keras.layers import LSTM, Dense,Flatten
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [3]:
#df_failures=pd.read_csv("https://raw.githubusercontent.com/ashishpatel26/Predictive_Maintenance_using_Machine-Learning_Microsoft_Casestudy/master/data/PdM_failures.csv")
#df_machines=pd.read_csv("https://raw.githubusercontent.com/ashishpatel26/Predictive_Maintenance_using_Machine-Learning_Microsoft_Casestudy/master/data/PdM_machines.csv")
#df_telemetry=pd.read_csv("https://raw.githubusercontent.com/ashishpatel26/Predictive_Maintenance_using_Machine-Learning_Microsoft_Casestudy/master/data/PdM_telemetry.csv")

#df_failures.to_csv('equipment_failures.csv')
#df_machines.to_csv('equipment_machines.csv')
#df_telemetry.to_csv('equipment_telemetry.csv')

In [4]:
df_failures=pd.read_csv('equipment_failures.csv')
df_machines=pd.read_csv('equipment_machines.csv')
df_telemetry=pd.read_csv('equipment_telemetry.csv')

In [29]:
#print(df_failures.head(1))
#print(df_machines.head(1))
#print(df_telemetry.head(1))
df=pd.merge(df_failures,df_machines, on="machineID")
df=pd.merge(df,df_telemetry,on="machineID")

df.drop(["Unnamed: 0","Unnamed: 0_x","Unnamed: 0_y","datetime_y","datetime_x"],axis=1,inplace=True)
df.reset_index(inplace=True)
df.rename(columns={"failure":"label"},inplace=True)

failure_types=set(df['label'])
#print(failure_types)

encoder=LabelEncoder()
df['label']=encoder.fit_transform(df['label'])
df['model']=encoder.fit_transform(df['model'])

X_columns=[x for x in df.columns if x!='label']

X=df[X_columns]
#X = np.asarray(X).astype(np.float32)
y=df['label']
#y = np.asarray(y).astype(np.float32)
y = pd.get_dummies(y)
print(y)

print(df.columns)
print(df.head(1))

         0  1  2  3
0        0  0  0  1
1        0  0  0  1
2        0  0  0  1
3        0  0  0  1
4        0  0  0  1
...     .. .. .. ..
6667116  0  1  0  0
6667117  0  1  0  0
6667118  0  1  0  0
6667119  0  1  0  0
6667120  0  1  0  0

[6667121 rows x 4 columns]
Index(['index', 'machineID', 'label', 'model', 'age', 'volt', 'rotate',
       'pressure', 'vibration'],
      dtype='object')
   index  machineID  label  model  age        volt      rotate    pressure  \
0      0          1      3      2   18  176.217853  418.504078  113.077935   

   vibration  
0  45.087686  


In [66]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)
#X_train = np.asarray(X_train).astype(np.float32)
#y_train = np.asarray(y_train).astype(np.float32)

In [67]:
model = Sequential()
#model.add(LSTM(200, activation='tanh', return_sequences=True,input_shape=(1, features)))
model.add(LSTM(200, return_sequences=True,input_shape = (X_train.shape[0],X_train.shape[1])))
model.add(Dropout(0.2))
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 50))
#model.add(Flatten())
model.add(Dense(len(y.columns),activation='softmax'))
history=model.compile(optimizer="rmsprop", loss='categorical_crossentropy', metrics=['accuracy'])

In [68]:
#X_train = np.resize(X_train,(X_train.shape[0],1,X_train.shape[1]))
model.fit(X_train,y_train,epochs=10)

Epoch 1/10


ValueError: in user code:

    File "C:\Users\dnishimoto\Anaconda3\envs\deeplearning\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\dnishimoto\Anaconda3\envs\deeplearning\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\dnishimoto\Anaconda3\envs\deeplearning\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\dnishimoto\Anaconda3\envs\deeplearning\lib\site-packages\keras\engine\training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\dnishimoto\Anaconda3\envs\deeplearning\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\dnishimoto\Anaconda3\envs\deeplearning\l

In [14]:
#grouped=df.groupby("label")
#batch_all_columns=['machineID',  'model', 'age', 'volt','rotate', 'pressure', 'vibration','label']
#batch_x_columns=['machineID',  'model', 'age', 'volt','rotate', 'pressure', 'vibration']



#print(X_columns)
#print(df[X_columns].head(10))
#print(pd.get_dummies(df['label']).head(10))
#for machineID in grouped:
#    label=machineID[0]
#    print(label)
#    df_batch=pd.DataFrame(columns=[batch_all_columns])
#    filter=df['label']==label
#    df_batch=pd.concat([df[filter][batch_all_columns],df_batch],ignore_index=True)
    #print(df_batch.columns)
    #df_batch.reset_index(inplace=True)
#    X=df_batch[batch_x_columns]
#    X = np.asarray(X).astype(np.float32)
#    X = np.resize(X,(X.shape[0],1,X.shape[1]))
#    y=df_batch['label']
#    print(X.shape[0],1,X.shape[1])
#    print("X:\n",X)
#    y=pd.get_dummies(df_batch['label'])
#   y = np.asarray(y).astype(np.float32)
    #y=df_batch['label'].apply(lambda x: 1 if x==1 else 0)
    
#    model.fit(X,y,batch_size=len(X),epochs=10)    
    #model.fit(X,y,epochs=10)    

0
1682112 1 1
X:
 [[[  1.         2.        18.       ... 418.5041   113.077934  45.087685]]

 [[  1.         2.        18.       ... 402.7475    95.460526  43.41397 ]]

 [[  1.         2.        18.       ... 527.34985   75.23791   34.17885 ]]

 ...

 [[100.         3.         5.       ... 447.81653   94.132835  48.31456 ]]

 [[100.         3.         5.       ... 413.77167  104.08107   44.83526 ]]

 [[100.         3.         5.       ... 496.09686   79.095535  37.845245]]]
Epoch 1/10


ValueError: in user code:

    File "C:\Users\dnishimoto\Anaconda3\envs\deeplearning\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\dnishimoto\Anaconda3\envs\deeplearning\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\dnishimoto\Anaconda3\envs\deeplearning\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\dnishimoto\Anaconda3\envs\deeplearning\lib\site-packages\keras\engine\training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\dnishimoto\Anaconda3\envs\deeplearning\lib\site-packages\keras\engine\training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "C:\Users\dnishimoto\Anaconda3\envs\deeplearning\lib\

In [ ]:
X=df[X_columns]
X = np.resize(X,(X.shape[0],1,X.shape[1]))
y_pred=model.predict(X)
print(y_pred)
print(history)